In [6]:
!pip install mmsegmentation
!pip install mmcv-full
!pip install sklearn

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for mmcv-full: filename=mmcv_full-1.7.0-cp36-cp36m-win_amd64.whl size=1286386 sha256=ce70d72760cd75809f2d78d7d5732bd5a0ddc2fa18c782c9c5633b7b2e27edb5
  Stored in directory: c:\users\mokom\appdata\local\pip\cache\wheels\4d\34\4d\a833f0a2b929e51d67f69d92f965279f350b57fc150542a880
Successfully built mmcv-full


In [30]:

import cv2
import numpy as np
import random
from pathlib import Path
import matplotlib.pyplot as plt
from mmseg.apis import init_segmentor, inference_segmentor
from sklearn.manifold import TSNE
import torchvision
import mmcv

In [17]:

config_file = 'https://github.com/open-mmlab/mmsegmentation/blob/master/configs/deeplabv3plus/deeplabv3plus_r50-d8_512x1024_40k_cityscapes.py'
checkpoint_file = 'https://download.openmmlab.com/mmsegmentation/v0.5/deeplabv3plus/deeplabv3plus_r50-d8_512x1024_40k_cityscapes/deeplabv3plus_r50-d8_512x1024_40k_cityscapes_20200605_094610-d222ffcd.pth'

In [22]:
!pip install wget
# google colaboで実行する場合は”wget $config_file”でOK
!python -m wget $config_file
!python -m wget $checkpoint_file


Saved under deeplabv3plus_r50-d8_512x1024_40k_cityscapes.py

Saved under deeplabv3plus_r50-d8_512x1024_40k_cityscapes_20200605_094610-d222ffcd.pth


SyntaxError: invalid syntax (<ipython-input-20-18f071420fbb>, line 1)

In [34]:
for key, val in config.items():
    print(key)
    print(val)
    print('---------------')

norm_cfg
{'type': 'BN', 'requires_grad': True}
---------------
model
{'type': 'EncoderDecoder', 'pretrained': 'open-mmlab://resnet50_v1c', 'backbone': {'type': 'ResNetV1c', 'depth': 50, 'num_stages': 4, 'out_indices': (0, 1, 2, 3), 'dilations': (1, 1, 2, 4), 'strides': (1, 2, 1, 1), 'norm_cfg': {'type': 'SyncBN', 'requires_grad': True}, 'norm_eval': False, 'style': 'pytorch', 'contract_dilation': True}, 'decode_head': {'type': 'DepthwiseSeparableASPPHead', 'in_channels': 2048, 'in_index': 3, 'channels': 512, 'dilations': (1, 12, 24, 36), 'c1_in_channels': 256, 'c1_channels': 48, 'dropout_ratio': 0.1, 'num_classes': 19, 'norm_cfg': {'type': 'SyncBN', 'requires_grad': True}, 'align_corners': False, 'loss_decode': {'type': 'CrossEntropyLoss', 'use_sigmoid': False, 'loss_weight': 1.0}}, 'auxiliary_head': {'type': 'FCNHead', 'in_channels': 1024, 'in_index': 2, 'channels': 256, 'num_convs': 1, 'concat_input': False, 'dropout_ratio': 0.1, 'num_classes': 19, 'norm_cfg': {'type': 'SyncBN', 'req

In [40]:
mmseg_path = Path('../../mmsegmentation')
config_file_path = mmseg_path / 'configs/deeplabv3plus/deeplabv3plus_r50-d8_512x1024_40k_cityscapes.py'
checkpoint_file_path = './deeplabv3plus_r50-d8_512x1024_40k_cityscapes_20200605_094610-d222ffcd.pth'
cfg = mmcv.Config.fromfile(config_file_path)

# avoid TypeError: init_segmentor() got an unexpected keyword argument 'norm_cfg'
cfg.norm_cfg = dict(type='BN', requires_grad=True)
cfg.model.backbone.norm_cfg = cfg.norm_cfg
cfg.model.decode_head.norm_cfg = cfg.norm_cfg
cfg.model.auxiliary_head.norm_cfg = cfg.norm_cfg
model = init_segmentor(cfg, checkpoint_file_path, device='cpu')

img = 'test.JPEG'
result = inference_segmentor(model, img)
model.show_result(img, result, show=True)

c:\Users\mokom\Anaconda3\envs\data_science\lib\site-packages\mmseg\models\losses\cross_entropy_loss.py:236: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  'Default ``avg_non_ignore`` is False, if you would like to '


load checkpoint from local path: ./deeplabv3plus_r50-d8_512x1024_40k_cityscapes_20200605_094610-d222ffcd.pth


c:\Users\mokom\Anaconda3\envs\data_science\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [55]:
def middle_layer_output(target, inputs):
    feature = None

    def forward_hook(module, inputs, outputs):
        global features
        features = outputs.detach().clone()

    handle = target.register_forward_hook(forward_hook)
    model.eval()
    inference_segmentor(model, inputs)
    handle.remove()
    return features

In [56]:
target = model.backbone.layer4[-1].conv3
middle_layer_output(target, img)

tensor([[[[ 0.0413,  0.0282,  0.0322,  ...,  0.0251,  0.0164,  0.0279],
          [ 0.0436,  0.0229,  0.0365,  ...,  0.0221,  0.0101,  0.0261],
          [ 0.0401,  0.0246,  0.0351,  ...,  0.0294,  0.0168,  0.0293],
          ...,
          [ 0.0241,  0.0201,  0.0254,  ...,  0.0296,  0.0280,  0.0334],
          [ 0.0261,  0.0204,  0.0220,  ...,  0.0323,  0.0271,  0.0309],
          [ 0.0222,  0.0175,  0.0217,  ...,  0.0259,  0.0178,  0.0254]],

         [[ 0.0320,  0.0264,  0.0326,  ...,  0.0314,  0.0302,  0.0330],
          [ 0.0340,  0.0345,  0.0351,  ...,  0.0264,  0.0272,  0.0288],
          [ 0.0342,  0.0298,  0.0356,  ...,  0.0284,  0.0292,  0.0278],
          ...,
          [ 0.0357,  0.0351,  0.0384,  ...,  0.0469,  0.0461,  0.0438],
          [ 0.0382,  0.0344,  0.0373,  ...,  0.0335,  0.0359,  0.0351],
          [ 0.0383,  0.0380,  0.0399,  ...,  0.0337,  0.0380,  0.0346]],

         [[ 0.0437,  0.0390,  0.0484,  ...,  0.0461,  0.0375,  0.0451],
          [ 0.0562,  0.0455,  